In [4]:
import pickle
import pandas as pd
import requests
import time
import json
from glob import glob

import seaborn as sns

In [5]:
dskey = pickle.load(open("darkskyapi.p",'rb'))
open("darkskyapi.p",'rb').close

date_vars = ['Date']
air_df = []
for file in glob('sfOzoneByYear/*.csv'):
    air_df.append(pd.read_csv(file, parse_dates=date_vars, date_parser=pd.to_datetime))


air_df = pd.concat(air_df)
air_df.shape

(3972, 20)

In [6]:
long = air_df.iloc[3666,-1]

lat =air_df.iloc[3666,-2]

day = air_df.iloc[3666,0]


day = int(time.mktime(day.timetuple()))
url ='https://api.darksky.net/forecast/'+dskey+'/'+str(lat)+','+str(long)+','+str(day)
weather= requests.get(url)
weather.json()['daily']['data'][0]

{'time': 1235635200,
 'summary': 'Drizzle in the morning.',
 'icon': 'rain',
 'sunriseTime': 1235659558,
 'sunsetTime': 1235700105,
 'moonPhase': 0.07,
 'precipIntensity': 0.0035,
 'precipIntensityMax': 0.0185,
 'precipIntensityMaxTime': 1235642400,
 'precipProbability': 0.74,
 'precipType': 'rain',
 'temperatureHigh': 59.01,
 'temperatureHighTime': 1235685600,
 'temperatureLow': 44.34,
 'temperatureLowTime': 1235743200,
 'apparentTemperatureHigh': 59.01,
 'apparentTemperatureHighTime': 1235685600,
 'apparentTemperatureLow': 41.81,
 'apparentTemperatureLowTime': 1235739600,
 'dewPoint': 47.07,
 'humidity': 0.8,
 'pressure': 1022.99,
 'windSpeed': 5.25,
 'windGust': 14.11,
 'windGustTime': 1235692800,
 'windBearing': 258,
 'cloudCover': 0.64,
 'uvIndex': 4,
 'uvIndexTime': 1235682000,
 'visibility': 9.872,
 'temperatureMin': 46.73,
 'temperatureMinTime': 1235721600,
 'temperatureMax': 59.01,
 'temperatureMaxTime': 1235685600,
 'apparentTemperatureMin': 45.51,
 'apparentTemperatureMinTim

In [7]:
weather

<Response [200]>

In [4]:
day_length =[]
temp_high =[]
temp_low =[]
dew_point =[]
humidity =[]
pressure =[]
wind_speed =[]
wind_bearing =[]
cloud_cover =[]
uv_index =[]

for i in range(air_df.shape[0]):
    try:
        long = air_df.iloc[i,-1]
        lat =air_df.iloc[i,-2]
        day = air_df.iloc[i,0]
        day = pd.to_datetime(day , format ='%m/%d/%Y')
        day = int(time.mktime(day.timetuple()))
        url ='https://api.darksky.net/forecast/'+dskey+'/'+str(lat)+','+str(long)+','+str(day)
        weather= requests.get(url)
        day_length.append(weather.json()['daily']['data'][0]['sunsetTime'] - weather.json()['daily']['data'][0]['sunriseTime'])
        try:
            temp_high.append(weather.json()['daily']['data'][0]['temperatureHigh'])
        except:
            temp_high.append(weather.json()['daily']['data'][0]['temperatureMax'])
        try:
            temp_low.append(weather.json()['daily']['data'][0]['temperatureLow'])
        except:
            temp_low.append(weather.json()['daily']['data'][0]['temperatureMin'])
        dew_point.append(weather.json()['daily']['data'][0]['dewPoint'])
        humidity.append(weather.json()['daily']['data'][0]['humidity'])
        pressure.append(weather.json()['daily']['data'][0]['pressure'])
        wind_speed.append(weather.json()['daily']['data'][0]['windSpeed'])
        wind_bearing.append(weather.json()['daily']['data'][0]['windBearing'])
        cloud_cover.append(weather.json()['daily']['data'][0]['cloudCover'])
        uv_index.append(weather.json()['daily']['data'][0]['uvIndex'])
    except:
        print('broken at: ' +str(i))

    

In [5]:
air_df['day_length'] = day_length
air_df['temp_high'] = temp_high
air_df['temp_low'] = temp_low
air_df['dew_point'] = dew_point
air_df['humidity'] = humidity
air_df['pressure'] = pressure
air_df['wind_speed'] = wind_speed
air_df['wind_bearing'] = wind_bearing
air_df['cloud_cover'] = cloud_cover
air_df['uv_index'] = uv_index

In [6]:
pickle.dump(air_df, open('air_df_1.p','wb'))

In [7]:
air_df = pickle.load(open("air_df_1.p",'rb'))
open('air_df_1.p','rb').close

In [8]:
df = air_df

df.columns

df.drop(columns=['Source','Site ID','POC','UNITS', 'DAILY_AQI_VALUE',
       'Site Name', 'DAILY_OBS_COUNT', 'PERCENT_COMPLETE',
       'AQS_PARAMETER_CODE', 'AQS_PARAMETER_DESC', 'CBSA_CODE', 'CBSA_NAME',
       'STATE_CODE', 'STATE', 'COUNTY_CODE', 'COUNTY', 'SITE_LATITUDE',
       'SITE_LONGITUDE'], inplace=True)
df.set_index('Date', inplace=True)

pickle.dump(df,open('ts_with_weather_sf.p', 'wb'))

df = pickle.load(open('ts_with_weather_sf.p', 'rb'))

In [9]:
df = pickle.load(open('ts_with_weather_sf.p', 'rb'))

In [10]:
df

,Daily Max 8-hour Ozone Concentration,day_length,temp_high,temp_low,dew_point,humidity,pressure,wind_speed,wind_bearing,cloud_cover,uv_index
Date,,,,,,,,,,,
2008-01-01,0.017,34563,54.21,43.21,35.67,0.63,1023.59,2.65,66,0.26,2
2008-01-02,0.024,34603,56.00,48.73,38.28,0.68,1018.58,0.56,115,0.44,2
2008-01-03,0.030,34646,56.67,52.35,45.36,0.77,1013.71,5.48,156,0.93,2
2008-01-04,0.036,34692,56.94,47.66,48.70,0.85,1002.50,7.34,186,0.87,2
2008-01-05,0.033,34741,51.05,42.85,42.72,0.80,1008.06,4.27,193,0.69,2
2008-01-06,0.027,34795,51.31,43.66,40.50,0.78,1011.78,1.86,193,0.71,2
2008-01-07,0.028,34850,51.56,43.43,39.25,0.76,1018.93,1.65,263,0.60,2
2008-01-08,0.018,34909,51.29,45.54,42.33,0.82,1019.51,3.91,164,0.83,2
2008-01-09,0.003,34971,49.73,47.60,44.80,0.89,1024.46,0.60,345,0.90,2
